In [ ]:
import sys
 
explicit_packages = [
    'tensorflow',
    'wandb',
    'numpy',
    'opencv-python-headless',  
    'imgaug'
]
 
indirect_packages = [
    'h5py',  
    'Pillow',  
    'matplotlib',  
    'scipy',  
    'scikit-learn'  
]
 
all_packages = explicit_packages + indirect_packages
 
package_versions = {}
for package in all_packages:
    try:
        imported_package = __import__(package)
        version = getattr(imported_package, '__version__', 'No version attribute')
        package_versions[package] = version
    except ImportError:
        package_versions[package] = 'Not installed'
 
package_versions

In [ ]:
import os
import subprocess

def get_system_info():
    print("--- System Information ---")
    print("OS:", os.uname().sysname, os.uname().release)
    print("Processor:", subprocess.check_output(['uname', '-m']).decode().strip())

def get_memory_info():
    print("\n--- Memory Information ---")
    meminfo = dict((i.split()[0].rstrip(':'),int(i.split()[1])) for i in open('/proc/meminfo').readlines())
    total_memory_kb = meminfo['MemTotal']
    available_memory_kb = meminfo['MemAvailable']
    print("Total Memory:", round(total_memory_kb / (1024.0 ** 2), 2), "GB")
    print("Available Memory:", round(available_memory_kb / (1024.0 ** 2), 2), "GB")

def get_gpu_info():
    print("\n--- GPU Information ---")
    try:
        gpu_info = subprocess.check_output(['nvidia-smi', '--query-gpu=memory.total,memory.used,memory.free,utilization.gpu,temperature.gpu', '--format=csv,noheader,nounits']).decode().strip().split('\n')
        for i, gpu in enumerate(gpu_info):
            print(f"GPU {i+1}:")
            total_mem, used_mem, free_mem, utilization, temp = gpu.split(', ')
            print("Memory Total:", total_mem, "MB")
            print("Memory Used:", used_mem, "MB")
            print("Memory Free:", free_mem, "MB")
            print("GPU Utilization:", utilization, "%")
            print("Temperature:", temp, "C")
            print()
    except subprocess.CalledProcessError:
        print("No NVIDIA GPU available.")

def main():
    get_system_info()
    get_memory_info()
    get_gpu_info()

if __name__ == "__main__":
    main()


In [ ]:
import os
import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  

if tf.__version__.startswith('2.'):
    tf.compat.v1.disable_eager_execution()

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  
config.log_device_placement = True  
sess = tf.compat.v1.Session(config=config)


In [ ]:
import platform
import psutil
#import cpuinfo
#import GPUtil

def get_system_info():
    print("--- System Information ---")
    print("OS:", platform.system(), platform.release())
    print("Processor:", platform.processor())

def get_memory_info():
    print("\n--- Memory Information ---")
    svmem = psutil.virtual_memory()
    print("Total Memory:", round(svmem.total / (1024.0 ** 3), 2), "GB")
    print("Available Memory:", round(svmem.available / (1024.0 ** 3), 2), "GB")

def get_gpu_info():
    print("\n--- GPU Information ---")
    gpus = GPUtil.getGPUs()
    if gpus:
        for i, gpu in enumerate(gpus):
            print(f"GPU {i+1}:")
            print("Name:", gpu.name)
            print("Memory Total:", gpu.memoryTotal, "MB")
            print("Memory Used:", gpu.memoryUsed, "MB")
            print("Memory Free:", gpu.memoryFree, "MB")
            print("GPU Utilization:", gpu.load * 100, "%")
            print("Temperature:", gpu.temperature, "C")
            print()
    else:
        print("No GPU available.")

def main():
    get_system_info()
#    get_cpu_info()
    get_memory_info()
#    get_gpu_info()

if __name__ == "__main__":
    main()


In [ ]:
import os
import subprocess

def get_system_info():
    print("--- System Information ---")
    print("OS:", os.uname().sysname, os.uname().release)
    print("Processor:", subprocess.check_output(['uname', '-m']).decode().strip())

def get_memory_info():
    print("\n--- Memory Information ---")
    meminfo = dict((i.split()[0].rstrip(':'),int(i.split()[1])) for i in open('/proc/meminfo').readlines())
    total_memory_kb = meminfo['MemTotal']
    available_memory_kb = meminfo['MemAvailable']
    print("Total Memory:", round(total_memory_kb / (1024.0 ** 2), 2), "GB")
    print("Available Memory:", round(available_memory_kb / (1024.0 ** 2), 2), "GB")

def main():
    get_system_info()
    get_memory_info()

if __name__ == "__main__":
    main()


In [ ]:
import os

def print_directory_structure(root_dir, depth=0, max_files=5):
    if depth > 0:  
        print("|  " * (depth-1) + "|--" + os.path.basename(root_dir))

    try:
        files = os.listdir(root_dir)
        for i, file in enumerate(files):
            if os.path.isdir(os.path.join(root_dir, file)):
                print_directory_structure(os.path.join(root_dir, file), depth + 1, max_files)
                if i >= max_files - 1:
                    break
            elif i < max_files:
                print("|  " * depth + "|--" + file)  
    except PermissionError:
        pass

root_dir = "../data_split/"

print_directory_structure(root_dir)


In [ ]:
import os

def check_data_integrity(dataset_dir):
    image_dir = os.path.join(dataset_dir, "images")
    mask_dir = os.path.join(dataset_dir, "masks")
    image_files = [f for f in os.listdir(image_dir) if f.endswith(".png")]
    mask_files = [f for f in os.listdir(mask_dir) if f.endswith(".png")]

    for image_file in image_files:
        mask_file = f"mask_{image_file}"
        if mask_file not in mask_files:
            print(f"Missing mask for image: {image_file}")



In [ ]:
check_data_integrity("../data_split/train")

In [ ]:
check_data_integrity("../data_split/val")


In [ ]:
class BuildingDataset(utils.Dataset):

    def load_mask(self, image_id):
        info = self.image_info[image_id]
        mask_path = info['mask_path']
        mask = skimage.io.imread(mask_path)

        print(f"Original mask shape for image_id {image_id}: {mask.shape}")

        if mask.ndim == 3:
            mask = skimage.color.rgb2gray(mask)
        
        mask = (mask > 0).astype(np.bool)

        mask = np.expand_dims(mask, axis=-1)

        print(f"Final mask shape for image_id {image_id}: {mask.shape}")
        class_ids = np.array([1 for _ in range(mask.shape[-1])])
        print(f"Class IDs for image_id {image_id}: {class_ids}")

        return mask, class_ids


In [ ]:
import tensorflow as tf
import h5py

print("TensorFlow version:", tf.__version__)

try:
    import keras
    print("Standalone Keras version:", keras.__version__)
except ImportError:
    print("Keras is integrated within TensorFlow, version:", tf.keras.__version__)

# Check h5py version
print("h5py version:", h5py.__version__)


In [ ]:
import tensorflow as tf

if tf.config.list_physical_devices('GPU'):
    print("TensorFlow can access GPUs.")
else:
    print("TensorFlow cannot access GPUs.")

print("Built with CUDA: ", tf.test.is_built_with_cuda())
print("CUDA GPUs available: ", tf.config.list_physical_devices('CUDA'))


In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

print("Built with CUDA:", tf.test.is_built_with_cuda())

if tf.test.gpu_device_name():
    print("Default GPU Device:", tf.test.gpu_device_name())
else:
    print("TensorFlow cannot access GPUs.")

import os
print("CUDA version (from environment variable):", os.getenv("CUDA_VERSION"))
print("cuDNN version (from environment variable):", os.getenv("CUDNN_VERSION"))


In [ ]:
import subprocess

cuda_version = subprocess.run(['nvcc', '--version'], text=True, capture_output=True)
print(cuda_version.stdout)

cudnn_version_cmd = "cat /usr/include/cudnn_version.h | grep CUDNN_MAJOR -A 2"
cudnn_version = subprocess.run(cudnn_version_cmd, text=True, capture_output=True, shell=True)
print(cudnn_version.stdout)

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
import subprocess

cuda_version = subprocess.check_output(["nvcc", "--version"]).decode("utf-8")
print("CUDA version:", cuda_version.split()[-2])

import tensorflow as tf
print(tf.keras.__version__)

In [ ]:
import numpy as np
from skimage.io import imread
import os

def load_mask(image_id, mask_dir):
    mask_path = os.path.join(mask_dir, f'{image_id}.png')  
    mask = imread(mask_path)
    
    unique_classes = np.unique(mask)[1:]  

    class_counts = {class_id: np.sum(mask == class_id) for class_id in unique_classes}
    
    return mask, class_counts

def print_mask_metrics(mask, class_counts):
    total_pixels = mask.size
    print(f"Total pixels in the mask: {total_pixels}")
    
    for class_id, count in class_counts.items():
        percentage = (count / total_pixels) * 100
        print(f"Class ID {class_id} covers {count} pixels, which is {percentage:.2f}% of the image.")

image_id = 'mask_chunk_19456_57856' 
mask_dir = '../data_split/train/masks/'  

mask, class_counts = load_mask(image_id, mask_dir)
print_mask_metrics(mask, class_counts)


In [ ]:
import os
import numpy as np
import skimage.io
import random

def analyze_mask_images(mask_dir, num_samples=50):
    analysis_results = []

    all_filenames = [f for f in os.listdir(mask_dir) if f.endswith(".png")]  

    sampled_filenames = random.sample(all_filenames, min(num_samples, len(all_filenames)))

    for filename in sampled_filenames:
        mask_path = os.path.join(mask_dir, filename)
        mask = skimage.io.imread(mask_path)

        unique, counts = np.unique(mask, return_counts=True)
        total_pixels = mask.size
        result = {
            'filename': filename,
            'unique_class_ids': unique,
            'class_pixel_counts': counts,
            'total_pixels': total_pixels
        }

        percentages = (counts / total_pixels) * 100
        result['class_percentages'] = percentages

        analysis_results.append(result)

    return analysis_results

mask_dir = '../data_split/train/masks/' 

results = analyze_mask_images(mask_dir, num_samples=50)

for result in results:
    print(f"Mask: {result['filename']}")
    for class_id, count, percentage in zip(result['unique_class_ids'], result['class_pixel_counts'], result['class_percentages']):
        print(f"  Class ID {class_id}: {count} pixels, {percentage:.2f}% of the image")
    print("-" * 50)


In [ ]:
import os
import sys
sys.path.insert(0, '/home/rk42218/Building_Detection/Mask_RCNN/env/lib/python3.7/site-packages/mask_rcnn_extracted/')
import numpy as np
import skimage.io
import skimage.color
#import wandb
import logging
import imgaug.augmenters as iaa
from keras.callbacks import Callback
from mrcnn.model import MaskRCNN
from mrcnn import utils
from mrcnn.config import Config
from mrcnn import model as modellib
import tensorflow as tf

import subprocess
import tensorflow as tf

import tensorflow as tf
from mrcnn.config import Config
from mrcnn import model as modellib, utils

import os
import sys
import numpy as np
import skimage.io
import skimage.color
import tensorflow as tf
from mrcnn.config import Config
from mrcnn import model as modellib, utils

import os
import sys
import subprocess
import logging
import numpy as np
import skimage.io
import skimage.color
import imgaug.augmenters as iaa
import tensorflow as tf
from tensorflow.keras.callbacks import Callback
from mrcnn.model import MaskRCNN  
from mrcnn import utils
from mrcnn.config import Config
from mrcnn import model as modellib
import os
import numpy as np
import tensorflow as tf
import skimage.io
import skimage.transform
import sys
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.utils import Progbar
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
import os
import json
import cv2
import numpy as np

def find_bbox(mask):
    """Find the bounding box of a mask. Returns an empty list if no object is found."""
    rows = np.any(mask, axis=1)
    cols = np.any(mask, axis=0)
    if not np.any(rows) or not np.any(cols):
        return [0, 0, 0, 0]  # Or return [] if you prefer
    
    y_min, y_max = np.where(rows)[0][[0, -1]]
    x_min, x_max = np.where(cols)[0][[0, -1]]
    return [int(x_min), int(y_min), int(x_max - x_min), int(y_max - y_min)]

def create_annotations(image_dir, mask_dir, output_file):
    annotations = {'images': []}

    for image_file in os.listdir(image_dir):
        image_id = os.path.splitext(image_file)[0]
        image_path = os.path.join(image_dir, image_file)

        class_id = 1  

        mask_file = f"mask_{image_id}.png"  
        mask_path = os.path.join(mask_dir, mask_file)

        if os.path.exists(mask_path):
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            bbox = find_bbox(mask)

            annotations['images'].append({
                'file_name': image_file,
                'id': image_id,
                'annotations': [
                    {
                        'class_id': class_id,
                        'bbox': bbox
                    }
                ]
            })

    with open(output_file, 'w') as f:
        json.dump(annotations, f, indent=4)

image_dir = '/home/rk42218/data_split/train/images/'
mask_dir = '/home/rk42218/data_split/train/masks'

output_file = '/home/rk42218/data_split/train/train_annotations.json'

create_annotations(image_dir, mask_dir, output_file)


In [ ]:
import os
import json
import cv2
import numpy as np

def find_bbox(mask):
    """Find the bounding box of a mask. Returns an empty list if no object is found."""
    rows = np.any(mask, axis=1)
    cols = np.any(mask, axis=0)
    if not np.any(rows) or not np.any(cols):
        return [0, 0, 0, 0]  # Or return [] if you prefer
    
    y_min, y_max = np.where(rows)[0][[0, -1]]
    x_min, x_max = np.where(cols)[0][[0, -1]]
    return [int(x_min), int(y_min), int(x_max - x_min), int(y_max - y_min)]

def create_annotations(image_dir, mask_dir, output_file):
    annotations = {'images': []}

    for image_file in os.listdir(image_dir):
        image_id = os.path.splitext(image_file)[0]
        image_path = os.path.join(image_dir, image_file)

        class_id = 1  

        mask_file = f"mask_{image_id}.png"  
        mask_path = os.path.join(mask_dir, mask_file)

        if os.path.exists(mask_path):
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            bbox = find_bbox(mask)

            annotations['images'].append({
                'file_name': image_file,
                'id': image_id,
                'annotations': [
                    {
                        'class_id': class_id,
                        'bbox': bbox
                    }
                ]
            })

    with open(output_file, 'w') as f:
        json.dump(annotations, f, indent=4)

image_dir = '/home/rk42218/data_split/val/images/'
mask_dir = '/home/rk42218/data_split/val/masks'

output_file = '/home/rk42218/data_split/val/val_annotations.json'

create_annotations(image_dir, mask_dir, output_file)
